In [1]:
import requests

def extract_game_id(week_data, team_id='252'):
    for game in week_data['events']:
        if game['competitions'][0]['competitors'][0]['team']['id'] == team_id or \
        game['competitions'][0]['competitors'][1]['team']['id'] == team_id:
            return game['id']
    return None

byu_game_ids = []

for week in range(14):
    params = {"week": week, "year": 2025, "seasontype": 2, "groups": 80}
    data = requests.get(
        "https://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard",
        params=params
    ).json()

    game_id = extract_game_id(data)
    if game_id:
        byu_game_ids.append(game_id)
byu_game_ids

['401756851',
 '401754526',
 '401756898',
 '401756903',
 '401756909',
 '401756916',
 '401756924',
 '401756931',
 '401756945',
 '401756950',
 '401756955']

In [2]:
all_game_info = []

for game_id in byu_game_ids:
    game_info = requests.get(
        f"https://site.api.espn.com/apis/site/v2/sports/football/college-football/summary?event={game_id}"
    ).json()
    all_game_info.append(game_info)

In [3]:
import pandas as pd

byu_data = {'byu_score': [], 
            'opponent_score': [], 
            'firstDowns': [], 
            'thirdDownEff': [], 
            'fourthDownEff': [], 
            'totalYards': [], 
            'netPassingYards': [], 
            'completionAttempts': [], 
            'yardsPerPass': [], 
            'rushingYards': [], 
            'rushingAttempts': [], 
            'yardsPerRushAttempt': [], 
            'totalPenaltiesYards': [], 
            'turnovers': [], 
            'fumblesLost': [], 
            'interceptions': [], 
            'possessionTime': []
}

for game_data in all_game_info:
    skip_row=False
    for team in game_data['header']['competitions'][0]['competitors']:
        if team['id'] == '252':
            try:
                byu_data['byu_score'].append(team['score'])
            except:
                skip_row = True
                continue
        else:
            try:
                byu_data['opponent_score'].append(team['score'])
            except:
                skip_row = True
                continue
    if skip_row:
        continue
    else:    
        for team in game_data['boxscore']['teams']:
            if team['team']['id'] == '252':
                for item in team['statistics']:
                    byu_data[item['name']].append(item['value'])

df = pd.DataFrame(byu_data)
df.drop(['totalYards', 'completionAttempts', 'totalPenaltiesYards', 'turnovers'], axis=1, inplace=True)
df

,byu_score,opponent_score,firstDowns,thirdDownEff,fourthDownEff,netPassingYards,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,fumblesLost,interceptions,possessionTime
0,69,0,28.0,0.600000,1.000000,138.0,7.667,468.0,48.0,9.750,1.0,0.0,2062
1,27,3,18.0,0.357143,0.500000,175.0,6.481,157.0,41.0,3.829,0.0,0.0,2096
2,34,13,20.0,0.500000,0.000000,246.0,9.462,172.0,34.0,5.059,0.0,0.0,1899
3,24,21,27.0,0.363636,0.666667,179.0,6.172,208.0,36.0,5.778,0.0,0.0,2015
4,38,24,23.0,0.333333,1.000000,351.0,14.040,165.0,42.0,3.929,2.0,1.0,1930
5,33,27,24.0,0.500000,0.666667,172.0,5.931,258.0,51.0,5.059,0.0,2.0,1967
6,24,21,22.0,0.250000,0.333333,166.0,6.917,202.0,42.0,4.810,0.0,0.0,1867
7,41,27,17.0,0.400000,1.000000,307.0,8.528,103.0,28.0,3.679,0.0,0.0,1670
8,7,29,14.0,0.214286,0.333333,188.0,4.947,67.0,27.0,2.481,2.0,1.0,1835


In [4]:
df.to_csv('byu_football_stats_2025.csv', index=False)